<a href="https://colab.research.google.com/github/AdrianduPlessis/UBI_Effects/blob/master/ML_Project_Proposal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hidden Effects of a UBI

Universal Basic Income (UBI) is not a new idea, I myself was introduced to the idea when Elon Musk first mentioned it in an interview back in 2017. Recently one of the democratic candidates, Andrew Yang, has been moving the idea closer to reality than it has ever been (on a massive scale, at least). With more attention than ever on UBI, some interesting opinions are being aired as to the possible social implications this massive economic disruption might have.  

Thinking through these issues myself, a lucky synapse fired reminding me of the Gini Index. The Gini Index is essentially a metric representing the inequality of income distribution within a certain geography. A UBI would obviously have the effect of lowering the Gini Index, most drastically on the bottom end. 
 
'Alright, so what?' I can hear you thinking. So this: The only reason I remember anything at all from the obscure YouTube video that introduced me to the Gini Index is because of the powerful correlation that exists between the Gini Index and adult_arrests rates. Anywhere on the spectrum... if you take two of the weathiest neighborhoods California, for example, and calculate the Gini Index for just those two neighborhoods, the neighborhood with the higher Gini Coefficient will (with some confidence) have the higher adult_arrests rate.  

'Neato, but so what?' Well, let me put it to you that using per county Gini Index, Income, and adult_arrests Rate data-- a decent ML model should be able to predict the reduction in adult_arrests rates of a given UBI ammount.  

**What's more:** that predicted reduction in adult_arrests-rate could be translated into reduced cost of incarseration (including burden of legal fees) to the government, information that would be very useful to a hopeful candidate trying to explain the value of the idea that their political campaign is based on.

#Hyp: income inequality -> adult_arrests


Hypothesis A: There exists a (statistically significant) correlation between Gini Coefficient (Income Inequality) and adult_arrestsrates within a specific county/state.

Determine p-values for individual counties' gini-to-adult_arrests over time.  
Determine p-values for counties' gini-to-adult_arrests for a given year.

##Tools of the trade

In [0]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt

##Import Datasets

In [0]:
gini_5yr_est_2017 = pd.read_csv('https://raw.githubusercontent.com/AdrianduPlessis/UBI_Effects/master/data/gini_est_5yr_17.csv', skiprows=1)

In [0]:
adult_arrests = pd.read_csv('https://raw.githubusercontent.com/AdrianduPlessis/UBI_Effects/master/data/ICPSR_36399/DS0001/36399-0001-Data.tsv', sep='\t')

In [0]:
FIPS = pd.read_csv('https://raw.githubusercontent.com/AdrianduPlessis/UBI_Effects/master/data/all-geocodes-v2016.xlsx%20-%20Sheet1.csv', skiprows=4)


###Data Wrangling

In [0]:
def change_id_to_fips(id):
  fips = int(id[-5:])
  return fips

In [0]:
gini_5yr_est_2017['Id'].apply(change_id_to_fips);

In [0]:
#Create dictionary to lookup State based on state FIPS
State_FIPS = FIPS[FIPS['County Code (FIPS)']==0]
State_FIPS = State_FIPS[State_FIPS['Summary Level']==40]

lookup_state_FIPS = dict(zip(State_FIPS['State Code (FIPS)'], State_FIPS['Area Name (including legal/statistical area description)']))

In [0]:
adult_arrests['state'] = adult_arrests['FIPS_ST'].map(lookup_state_FIPS)

In [0]:
def combine_cty_st_fips(cty, st):
  full_fips = st*1000 + cty
  return full_fips

In [0]:
FIPS['FIPS'] = combine_cty_st_fips(FIPS['County Code (FIPS)'], FIPS['State Code (FIPS)'])
adult_arrests['FIPS'] = combine_cty_st_fips(adult_arrests['FIPS_CTY'], adult_arrests['FIPS_ST'])

##Merge Datasets

In [0]:
#Note: missing gini data for 35 counties (based on shapes of dfs being merged)
arrest_by_ineq = adult_arrests.merge(gini_5yr_est_2017, left_on='FIPS', right_on='Id2')

###Further Wrangling

In [0]:
#Engineer feature that controls for variation in population size
arrest_by_ineq['arrests_per_person'] = arrest_by_ineq['CPOPARST']/ arrest_by_ineq['GRNDTOT']

In [0]:
#TODO: Control all arrest type for population size

In [14]:
arrest_by_ineq.shape

(3134, 64)

In [0]:
#Drop NaNs
#Stretch: Impute
arrest_by_ineq = arrest_by_ineq.dropna()

In [16]:
arrest_by_ineq.shape

(3129, 64)

In [0]:
#Had to drop 5 counties/observations

In [18]:
arrest_by_ineq['Estimate; Gini Index'].corr(arrest_by_ineq['arrests_per_person'])

nan

##Predict adult_arrests on gini coefficient (Baseline)

#Predict future gini values (optional)

#Apply UBI transform to Gini values

#Predict adult_arrests rate without UBI

#Predict adult_arrests rate with UBI

#Additional Questions:  


##Correlation between income metric and trend in adult_arrests rate?  
positive feedback loop?

##Mean/Median income predictive of type of crime?

#TODO 
11/16/19:  
Merge relevant adult_arrests data.  
Collapse by county.  
